In [ ]:
#Checks if yahoo finance package is installed
!pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# Load the CSV file with stock data, skipping the last two rows
csv_file = 'My portfolio - Mystk 2022.csv'  # Update with your CSV file path
stock_data = pd.read_csv(csv_file, skipfooter=2, engine='python')

# Drop rows with NaN in 'Stock ETF' and ensure the column is treated as string
stock_data = stock_data.dropna(subset=['Stock ETF'])
stock_data['Stock ETF'] = stock_data['Stock ETF'].astype(str)

# Remove dollar sign from 'Cost/share' and convert to numeric
stock_data['Cost/share'] = stock_data['Cost/share'].replace('[\$,]', '', regex=True).astype(float)

# Remove commas from 'No. of shares' and convert to numeric
stock_data['No. of shares'] = stock_data['No. of shares'].replace(',', '', regex=True).astype(float)

# Replace NaN values in 'Cost/share' and 'No. of shares' with 0
stock_data['Cost/share'] = pd.to_numeric(stock_data['Cost/share'], errors='coerce').fillna(0)
stock_data['No. of shares'] = pd.to_numeric(stock_data['No. of shares'], errors='coerce').fillna(0)

# Extract information from the CSV file
tickers = stock_data['Stock ETF'].tolist()
cost_per_share = dict(zip(stock_data['Stock ETF'], stock_data['Cost/share']))
number_of_shares = dict(zip(stock_data['Stock ETF'], stock_data['No. of shares']))

# Create an empty list to store data
data = []

# Loop through tickers and gather data
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="ytd")  # Fetch YTD data

        if hist.empty:
            print(f"{ticker}: No data found, symbol may be delisted")
            continue

        current_price = hist['Close'][-1]
        prev_close = hist['Close'][-2]
        change = current_price - prev_close
        min_100_day = hist['Close'].tail(100).min()
        max_100_day = hist['Close'].tail(100).max()
        ma_100_day = hist['Close'].tail(100).mean()
        ma_50_day = hist['Close'].tail(50).mean()
        pct_change_month = (hist['Close'][-1] - hist['Close'][-21]) / hist['Close'][-21] * 100 if len(hist) >= 21 else np.nan
        pe_ratio = stock.info.get('forwardPE', np.nan)
        peg_ratio = stock.info.get('pegRatio', np.nan)
        dividend_yield = stock.info.get('dividendYield', np.nan)
        sector = stock.info.get('sector', 'Unknown')
        
        # Retrieve values and handle missing data
        cost_per_share_ticker = cost_per_share.get(ticker, 0)
        number_of_shares_ticker = number_of_shares.get(ticker, 0)
        
        # Calculate overall stock gain
        overall_stock_gain = (current_price - cost_per_share_ticker) * number_of_shares_ticker

        # Append data to the list
        data.append({
            "Stock Ticker": ticker,
            "Current Price": current_price,
            "Previous Closing Price": prev_close,
            "Change": change,
            "100 Day Minimum": min_100_day,
            "100 Day Maximum": max_100_day,
            "100 Day Moving Average": ma_100_day,
            "50 Day Moving Average": ma_50_day,
            "Percent Change Over a Month": pct_change_month,
            "Price to Earnings": pe_ratio,
            "Price to Earnings Growth": peg_ratio,
            "Yield": dividend_yield,
            "Sector Information": sector,
            "Cost Per Share": cost_per_share_ticker,
            "Number of Shares Owned": number_of_shares_ticker,
            "Overall Stock Gain": overall_stock_gain
        })
    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Check if any data was collected before attempting to create a DataFrame
if data:
    # Create DataFrame from the list of data
    df = pd.DataFrame(data)

    # Save DataFrame to a CSV file
    csv_filename = 'stock_data.csv'
    df.to_csv(csv_filename, index=False)
else:
    print("No valid data collected.")

In [ ]:
# Display the DataFrame
df

In [ ]:
# Lists to store categorized stocks
below_50 = []
below_both = []
above_50 = []
above_both = []

# Loop through tickers and categorize based on moving averages
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="ytd")  # Fetch YTD data

        if hist.empty:
            print(f"{ticker}: No data found, symbol may be delisted")
            continue

        current_price = hist['Close'][-1]
        ma_50_day = hist['Close'].tail(50).mean()
        ma_100_day = hist['Close'].tail(100).mean()

        # Categorize based on current price and moving averages
        if current_price < ma_50_day:
            below_50.append(ticker)
            if current_price < ma_100_day:
                below_both.append(ticker)
        if current_price > ma_50_day:
            above_50.append(ticker)
            if current_price > ma_100_day:
                above_both.append(ticker)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        
print("Stocks below 50-day moving average:")
print(below_50)

print("Stocks below both 50 and 100-day moving averages:")
print(below_both)

print("Stocks above 50-day moving average:")
print(above_50)

print("Stocks above both 50 and 100-day moving averages:")
print(above_both)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Dictionary to store sector information and daily closing prices
sector_closing_prices = defaultdict(lambda: defaultdict(float))

# Loop through tickers to gather sector information and daily closing prices
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1mo")  # Fetch 1 month of data

        if hist.empty or len(hist) < 2:  # Skip if less than 2 days of data
            print(f"{ticker}: No sufficient data found, symbol may be delisted")
            continue

        sector = stock.info.get('sector', 'Unknown')
        if sector == 'Unknown':
            continue  # Skip if sector information is unknown

        # Sum closing prices for each sector and date
        for date, close in hist['Close'].items():
            sector_closing_prices[sector][date] += close

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Create a separate graph for each sector
for sector, daily_closing_prices in sector_closing_prices.items():
    plt.figure(figsize=(7, 3))
    # Extract dates and total closing prices
    dates, total_closing_prices = zip(*sorted(daily_closing_prices.items()))
    plt.plot(dates, total_closing_prices, marker='o', linestyle='-', label=sector)

    plt.xlabel('Date')
    plt.ylabel('Total Closing Price ($)')
    plt.title(f'Total Closing Price Over the Past Month for {sector}')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Dictionary to store sector information and monthly changes
sector_info = defaultdict(list)

# Loop through tickers to gather sector information and calculate monthly percent change
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1mo")  # Fetch 1 month of data

        if hist.empty or len(hist) < 2:  # Skip if less than 2 days of data
            print(f"{ticker}: No sufficient data found, symbol may be delisted")
            continue

        sector = stock.info.get('sector', 'Unknown')
        if sector == 'Unknown':
            continue  # Skip if sector information is unknown

        # Calculate percent change over the past month
        pct_change_month = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0] * 100

        # Add ticker's percent change to the sector_info dictionary
        sector_info[sector].append(pct_change_month)

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Calculate the average percent change for each sector
sector_percent_changes = {sector: np.mean(changes) for sector, changes in sector_info.items()}

# Convert to DataFrame for tabular display
df_sector_changes = pd.DataFrame(list(sector_percent_changes.items()), columns=['Sector', 'Monthly Percent Change'])

# Display the DataFrame
print("Monthly Percent Change by Sector:")
df_sector_changes

# Optionally, you can save the table to a CSV file
# df_sector_changes.to_csv('sector_percent_changes.csv', index=False)

In [ ]:
# Filter sectors with a percent change greater than 10% in either direction
significant_changes = df_sector_changes[abs(df_sector_changes['Monthly Percent Change']) > 10]

# Display the significant changes
print("\nSectors with Monthly Percent Change Greater Than 10%:")
significant_changes

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

def plot_stock_history(ticker, start_date=None):
    stock = yf.Ticker(ticker)
    
    # Fetch data from start_date to the current date or YTD
    if start_date:
        hist = stock.history(start=start_date)
    else:
        hist = stock.history(period="ytd")

    if hist.empty:
        print(f"{ticker}: No data found, symbol may be delisted or data may be unavailable.")
        return

    # Plot the closing price
    plt.figure(figsize=(7, 3))
    plt.plot(hist.index, hist['Close'], label='Closing Price', color='blue')
    plt.title(f'{ticker} Stock Price History')
    plt.xlabel('Date')
    plt.ylabel('Closing Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

def main():
    while True:
        # Ask if the user wants a graph
        user_input = input("Would you like a graph of the price history of a stock? (yes/no): ").strip().lower()
        if user_input == 'yes':
            ticker = input("Enter the stock ticker: ").strip().upper()
            date_choice = input("Would you like the data from a specific start date or year-to-date? (start_date/ytd): ").strip().lower()

            if date_choice == 'start_date':
                start_date = input("Enter the start date (YYYY-MM-DD): ").strip()
                try:
                    # Validate date format
                    datetime.strptime(start_date, "%Y-%m-%d")
                except ValueError:
                    print("Invalid date format. Please use YYYY-MM-DD.")
                    continue
            elif date_choice == 'ytd':
                start_date = None
            else:
                print("Invalid choice. Please enter 'start_date' or 'ytd'.")
                continue

            plot_stock_history(ticker, start_date)
        elif user_input == 'no':
            print("Exiting the program.")
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")

if __name__ == "__main__":
    main()


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Function to calculate percent change over a specified period
def calculate_percent_change(hist, period):
    if len(hist) < period:
        return np.nan
    return (hist['Close'][-1] - hist['Close'][-period]) / hist['Close'][-period] * 100

# Dictionary to store sector information and percent changes for different periods
sector_percent_changes = defaultdict(lambda: {'1mo': [], '6mo': [], '1yr': [], 'ytd': []})

# Loop through tickers to gather sector information and calculate percent changes
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist_ytd = stock.history(period="1y")  # Fetch 1-year data

        if hist_ytd.empty or len(hist_ytd) < 2:
            print(f"{ticker}: No sufficient data found, symbol may be delisted")
            continue

        sector = stock.info.get('sector', 'Unknown')
        if sector == 'Unknown':
            continue  # Skip if sector information is unknown

        # Calculate percent changes
        pct_change_1mo = calculate_percent_change(hist_ytd, 21)   # Approx. 21 trading days in a month
        pct_change_6mo = calculate_percent_change(hist_ytd, 126)  # Approx. 126 trading days in 6 months
        pct_change_1yr = calculate_percent_change(hist_ytd, 252)  # Approx. 252 trading days in a year
        pct_change_ytd = (hist_ytd['Close'][-1] - hist_ytd['Close'][0]) / hist_ytd['Close'][0] * 100

        # Store the percent changes in the sector_percent_changes dictionary
        sector_percent_changes[sector]['1mo'].append(pct_change_1mo)
        sector_percent_changes[sector]['6mo'].append(pct_change_6mo)
        sector_percent_changes[sector]['1yr'].append(pct_change_1yr)
        sector_percent_changes[sector]['ytd'].append(pct_change_ytd)

    except Exception as e:
        print(f"Error processing {ticker}: {e}")

# Calculate the average percent change for each sector and period
average_sector_changes = {sector: {period: np.nanmean(changes) for period, changes in periods.items()}
                          for sector, periods in sector_percent_changes.items()}

# Create separate bar graphs for each sector
for sector, changes in average_sector_changes.items():
    plt.figure(figsize=(7, 3))
    periods = list(changes.keys())
    values = list(changes.values())
    
    max_val = max(values) * 1.05  # 5% more than the maximum value
    min_val = min(values) * 0.95  # 5% less than the minimum value
    
    plt.bar(periods, values, color=['blue', 'orange', 'green', 'red'])
    plt.title(f'Percent Change for {sector}')
    plt.ylabel('Percent Change (%)')
    plt.xlabel('Period')
    plt.ylim([min_val, max_val])  # Dynamic y-axis limits
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Assuming df is the DataFrame containing the stock data with sector information.
# Define sector ETFs or major indexes for each sector
sector_etfs = {
    'Technology': 'XLC',  # Communication Services Select Sector SPDR Fund
    'Healthcare': 'XLB',  # Health Care Select Sector SPDR Fund
    'Financials': 'XLF',  # Financial Select Sector SPDR Fund
    'Consumer Discretionary': 'XLY',  # Consumer Discretionary Select Sector SPDR Fund
    'Energy': 'XLE',  # Energy Select Sector SPDR Fund
    'Real Estate': 'XHB',  # SPDR Homebuilders ETF (covers real estate sector)
    'Industrials': 'XLI',  # Industrial Select Sector SPDR Fund
    'Financial Services': 'XLF',  # Financial Select Sector SPDR Fund (same as Financials)
    'Consumer Cyclical': 'XLY',  # Consumer Discretionary Select Sector SPDR Fund (same as Consumer Discretionary)
    'Communication Services': 'XLC',  # Communication Services Select Sector SPDR Fund
}

# Step 1: Group by sector
sectors = df.groupby('Sector Information')

# Step 2: Calculate daily percent change for owned stocks and sector ETFs
sector_changes = {}
sector_data = {}
for sector, data in sectors:
    tickers = data['Stock Ticker'].unique()
    tickers_str = ' '.join(tickers)  # Convert array to a string
    
    # Download historical data for your owned stocks
    historical_data = yf.download(tickers_str, period='1mo')
    closing_prices = historical_data['Close']
    
    # Calculate daily percent change for owned stocks
    if isinstance(closing_prices, pd.DataFrame):
        daily_percent_change_stocks = closing_prices.pct_change().mean(axis=1)
    else:  # If closing_prices is a Series
        daily_percent_change_stocks = closing_prices.pct_change()
    
    # Store daily percent change for owned stocks
    sector_changes[sector] = daily_percent_change_stocks
    
    # Download historical data for sector ETF
    sector_etf = sector_etfs.get(sector, None)
    if sector_etf:
        sector_data[sector] = yf.download(sector_etf, period='1mo')['Close'].pct_change()

# Step 3: Generate line graphs for each sector
for sector, daily_change_stocks in sector_changes.items():
    plt.figure(figsize=(10, 3))
    
    # Plot daily percent change of owned stocks
    plt.plot(daily_change_stocks.index, daily_change_stocks, label=f"{sector} Owned Stocks", color='blue')
    
    # Plot daily percent change of sector ETF
    if sector in sector_data:
        daily_change_sector = sector_data[sector]
        plt.plot(daily_change_sector.index, daily_change_sector, label=f"{sector} Sector", color='red')
    
    plt.title(f"Daily Percent Change - {sector} Sector")
    plt.xlabel("Date")
    plt.ylabel("Percent Change")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Define sector ETFs or major indexes for each sector
sector_etfs = {
    'Technology': 'XLC',
    'Healthcare': 'XLB',
    'Financials': 'XLF',
    'Consumer Discretionary': 'XLY',
    'Energy': 'XLE',
    'Real Estate': 'XHB',
    'Industrials': 'XLI',
    'Financial Services': 'XLF',  # same as Financials
    'Consumer Cyclical': 'XLY',  # same as Consumer Discretionary
    'Communication Services': 'XLC',  # same as Technology
}

# Ask user for the specific stock ticker
stock_ticker = input("Enter the stock ticker you want to analyze: ").upper()

# Assuming df is the DataFrame containing the stock data with sector information.
# Step 1: Find the sector of the specific stock
stock_data = df[df['Stock Ticker'] == stock_ticker]
if stock_data.empty:
    print(f"Stock ticker {stock_ticker} not found in the DataFrame.")
else:
    sector = stock_data['Sector Information'].values[0]
    sector_etf = sector_etfs.get(sector, None)
    
    if not sector_etf:
        print(f"No sector ETF found for {sector}.")
    else:
        # Download historical data for the specific stock
        stock_data = yf.download(stock_ticker, period='1mo')['Close']
        daily_percent_change_stock = stock_data.pct_change()
        
        # Download historical data for the sector ETF
        sector_data = yf.download(sector_etf, period='1mo')['Close']
        daily_percent_change_sector = sector_data.pct_change()
        
        # Plot the comparison
        plt.figure(figsize=(10, 3))
        plt.plot(daily_percent_change_stock.index, daily_percent_change_stock, label=f"{stock_ticker} Daily Change", color='blue')
        plt.plot(daily_percent_change_sector.index, daily_percent_change_sector, label=f"{sector} Sector Daily Change", color='red')
        plt.title(f"Daily Percent Change - {stock_ticker} vs {sector} Sector")
        plt.xlabel("Date")
        plt.ylabel("Percent Change")
        plt.legend()
        plt.grid(True)
        plt.show()
